In [69]:
# Data handling
import numpy as np
import pandas as pd
from IPython.display import display

from sklearn.metrics import (precision_score, recall_score, classification_report, accuracy_score, hamming_loss, f1_score,
    jaccard_score,
    roc_auc_score,
    average_precision_score,
    matthews_corrcoef,
    cohen_kappa_score
)


# Signal processing
import scipy.signal as signal

# Feature extraction
import neurokit2 as nk     # for ECG feature extraction (heart rate, peaks, etc.)

# Visualization
import matplotlib.pyplot as plt
import json

import matplotlib.pyplot as plt
from challenge import load_header, get_labels

In [33]:

classes = ['6374002', '10370003', '17338001', '39732003', '47665007', '59118001', '59931005',
                '111975006', '164889003', '164890007', '164909002', '164917005', '164934002',
                '164947007', '251146004', '270492004', '284470004', '365413008', '426177001', '426627000',
                '426783006', '427084000', '427393009', '445118002', '698252002', '713426002']


def clean_labels(header):
    classes_from_header = get_labels(header)
    
    if '733534002' in classes_from_header:
        classes_from_header[classes_from_header.index('733534002')] = '164909002'
        classes_from_header = list(set(classes_from_header))
    if '713427006' in classes_from_header:
        classes_from_header[classes_from_header.index('713427006')] = '59118001'
        classes_from_header = list(set(classes_from_header))
    if '63593006' in classes_from_header:
        classes_from_header[classes_from_header.index('63593006')] = '284470004'
        classes_from_header = list(set(classes_from_header))
    if '427172004' in classes_from_header:
        classes_from_header[classes_from_header.index('427172004')] = '17338001'
        classes_from_header = list(set(classes_from_header))
    
    local_label = np.zeros((26,), dtype=bool)
    for label in classes_from_header:
        if label in classes:
            j = classes.index(label)
            local_label[j] = True

    
    return classes_from_header, local_label


In [34]:
with open("results/ALPHA_ONLY_NEW_DOMAIN_KNOWLEDGE_12_LSTM_7_2_training_and_test/2025-01-12/fold_0.json", "r") as f:
    results = json.load(f)
    print(results.keys())

print(len(results["labels"]))
print(results["labels"][0])

dict_keys(['c', 'labels', 'binary_outputs', 'scalar_outputs', 'times', 'auroc', 'auprc', 'auroc_classes', 'auprc_classes', 'f_measure', 'f_measure_classes', 'challenge_metric', 'leads', 'fold', 'experiment', 'network', 'accuracy'])
15021
[False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, True, False]


In [35]:
test_filename = "h5_datasets/3s_full/cinc_database_test_('I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6')_0.h5_header_recording_files.json"
with open(test_filename, 'r') as f:
    loaded_dict = json.load(f)
    print(len(loaded_dict['header_files']))
    print(len(loaded_dict['recording_files']))


header = load_header(loaded_dict['header_files'][0])
current_labels, labels_bin= clean_labels(header)
print(current_labels)
print(labels_bin)

15021
15021
['164889003', '251146004']
[False False False False False False False False  True False False False
 False False  True False False False False False False False False False
 False False]


In [37]:
def load_info_for_single_fold(fold: int, experiment_results_path: str):
    with open(f"{experiment_results_path}/fold_{fold}.json", "r") as f:
        results = json.load(f)

    test_filename = f"h5_datasets/3s_full/cinc_database_test_('I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6')_{fold}.h5_header_recording_files.json"
    with open(test_filename, 'r') as f:
        loaded_dict = json.load(f)

    ground_truth_labels = []
    total_length = len(loaded_dict['header_files'])
    one_percent_progress = total_length // 100
    for idx, header_file in enumerate(loaded_dict['header_files']):
        header = load_header(header_file)
        current_labels, labels_bin= clean_labels(header)
        ground_truth_labels.append(labels_bin)
    
    results["ground_truth"] = ground_truth_labels
    return results
                
    

In [38]:
single_fold_full_results = load_info_for_single_fold(0, "results/ALPHA_ONLY_NEW_DOMAIN_KNOWLEDGE_12_LSTM_7_2_training_and_test/2025-01-12")
single_fold_full_results.keys()


dict_keys(['c', 'labels', 'binary_outputs', 'scalar_outputs', 'times', 'auroc', 'auprc', 'auroc_classes', 'auprc_classes', 'f_measure', 'f_measure_classes', 'challenge_metric', 'leads', 'fold', 'experiment', 'network', 'accuracy', 'ground_truth'])

In [67]:
def calculate_all_measures(y_true, y_pred, class_names):
    precision = precision_score(y_true, y_pred, average=None)
    recall = recall_score(y_true, y_pred, average=None)
    accuracy = accuracy_score(y_true, y_pred)
    h_loss = hamming_loss(y_true, y_pred)
    hamming_acc = 1 - h_loss
    f1 = f1_score(y_true, y_pred, average=None)
    jaccard = jaccard_score(y_true, y_pred, average=None)
    roc_auc = roc_auc_score(y_true, y_pred, average=None)
    avg_prec = average_precision_score(y_true, y_pred, average=None)
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    mcc = [matthews_corrcoef(y_true[:, i], y_pred[:, i]) for i in range(y_true.shape[1])]
    support = y_true.sum(axis=0)  # Number of true instances per class

    
    df = pd.DataFrame({
    'Precision'         : precision,
    'Recall'            : recall,
    'Accurracy'         : accuracy,
    'Hamming Loss'      : h_loss,
    'Hamming Acc'       : hamming_acc,
    'F1-Score'          : f1,
    'Jaccard (IoU)'     : jaccard,
    'ROC AUC'           : roc_auc,
    'Avg Precision AUC' : avg_prec,
    'MCC'               : mcc,
    'Support'           : support
    }, index=class_names)
    return df

In [48]:
y_true = single_fold_full_results['ground_truth']
y_pred = single_fold_full_results['labels']

In [70]:
branches = {
    "alpha": "results/ALPHA_ONLY_NEW_DOMAIN_KNOWLEDGE_12_LSTM_7_2_training_and_test/2025-01-12",
    "beta": "results/BETA_ONLY_NEW_DOMAIN_KNOWLEDGE_12_LSTM_7_2_training_and_test/2025-01-13",
    "gamma": "results/GAMMA_ONLY_NEW_DOMAIN_KNOWLEDGE_12_LSTM_7_2_training_and_test/2025-01-14",
    "delta": "results/DELTA_ONLY_NEW_DOMAIN_KNOWLEDGE_12_LSTM_7_2_training_and_test/2025-01-14",
    "epsilon": "results/EPSILON_ONLY_NEW_DOMAIN_KNOWLEDGE_12_LSTM_7_2_training_and_test/2025-01-14",
    "zeta": "results/ZETA_ONLY_NEW_DOMAIN_KNOWLEDGE_12_LSTM_7_2_training_and_test/2025-01-16"
    }
final_dict = {}
for branch_name, path in branches.items():
    dfs = []
    for fold in range(5):
        fold_results = load_info_for_single_fold(fold, path)
        y_true = fold_results['ground_truth']
        y_pred = fold_results['labels']
        dfs.append(calculate_all_measures(y_true, y_pred, classes))

    df_avg = sum(dfs) / len(dfs)
    display(df_avg)

    final_dict[branch_name] = df_avg
        
        
        
        
    

,Precision,Recall,Accurracy,Hamming Loss,Hamming Acc,F1-Score,Jaccard (IoU),ROC AUC,Avg Precision AUC,MCC,Support
6374002,0.009945,0.084473,0.008122,0.109312,0.890688,0.017791,0.008977,0.512156,0.007397,0.008519,107.2
10370003,0.019537,0.110488,0.008122,0.109312,0.890688,0.033200,0.016884,0.504181,0.018284,0.003663,271.8
17338001,0.017085,0.004470,0.008122,0.109312,0.890688,0.007079,0.003554,0.499022,0.023919,-0.003674,359.6
39732003,0.074427,0.003622,0.008122,0.109312,0.890688,0.006905,0.003467,0.500029,0.077174,-0.000143,1158.6
47665007,0.021254,0.022175,0.008122,0.109312,0.890688,0.021685,0.010964,0.503019,0.015806,0.005851,235.4
59118001,0.067198,0.057674,0.008122,0.109312,0.890688,0.062055,0.032023,0.502855,0.061285,0.006151,915.0
59931005,0.052497,0.040541,0.008122,0.109312,0.890688,0.045738,0.023409,0.500131,0.052291,0.000257,785.0
111975006,0.020341,0.013048,0.008122,0.109312,0.890688,0.015878,0.008018,0.498645,0.024801,-0.003502,372.4
164889003,0.063282,0.064158,0.008122,0.109312,0.890688,0.063716,0.032920,0.499576,0.064080,-0.000852,962.6
164890007,0.109081,0.013205,0.008122,0.109312,0.890688,0.023548,0.011917,0.499952,0.110114,-0.000322,1653.8


,Precision,Recall,Accurracy,Hamming Loss,Hamming Acc,F1-Score,Jaccard (IoU),ROC AUC,Avg Precision AUC,MCC,Support
6374002,0.009945,0.084473,0.008122,0.109312,0.890688,0.017791,0.008977,0.512156,0.007397,0.008519,107.2
10370003,0.019537,0.110488,0.008122,0.109312,0.890688,0.033200,0.016884,0.504181,0.018284,0.003663,271.8
17338001,0.017085,0.004470,0.008122,0.109312,0.890688,0.007079,0.003554,0.499022,0.023919,-0.003674,359.6
39732003,0.074427,0.003622,0.008122,0.109312,0.890688,0.006905,0.003467,0.500029,0.077174,-0.000143,1158.6
47665007,0.021254,0.022175,0.008122,0.109312,0.890688,0.021685,0.010964,0.503019,0.015806,0.005851,235.4
59118001,0.067198,0.057674,0.008122,0.109312,0.890688,0.062055,0.032023,0.502855,0.061285,0.006151,915.0
59931005,0.052497,0.040541,0.008122,0.109312,0.890688,0.045738,0.023409,0.500131,0.052291,0.000257,785.0
111975006,0.020341,0.013048,0.008122,0.109312,0.890688,0.015878,0.008018,0.498645,0.024801,-0.003502,372.4
164889003,0.063282,0.064158,0.008122,0.109312,0.890688,0.063716,0.032920,0.499576,0.064080,-0.000852,962.6
164890007,0.109081,0.013205,0.008122,0.109312,0.890688,0.023548,0.011917,0.499952,0.110114,-0.000322,1653.8


,Precision,Recall,Accurracy,Hamming Loss,Hamming Acc,F1-Score,Jaccard (IoU),ROC AUC,Avg Precision AUC,MCC,Support
6374002,0.009945,0.084473,0.008122,0.109312,0.890688,0.017791,0.008977,0.512156,0.007397,0.008519,107.2
10370003,0.019537,0.110488,0.008122,0.109312,0.890688,0.033200,0.016884,0.504181,0.018284,0.003663,271.8
17338001,0.017085,0.004470,0.008122,0.109312,0.890688,0.007079,0.003554,0.499022,0.023919,-0.003674,359.6
39732003,0.074427,0.003622,0.008122,0.109312,0.890688,0.006905,0.003467,0.500029,0.077174,-0.000143,1158.6
47665007,0.021254,0.022175,0.008122,0.109312,0.890688,0.021685,0.010964,0.503019,0.015806,0.005851,235.4
59118001,0.067198,0.057674,0.008122,0.109312,0.890688,0.062055,0.032023,0.502855,0.061285,0.006151,915.0
59931005,0.052497,0.040541,0.008122,0.109312,0.890688,0.045738,0.023409,0.500131,0.052291,0.000257,785.0
111975006,0.020341,0.013048,0.008122,0.109312,0.890688,0.015878,0.008018,0.498645,0.024801,-0.003502,372.4
164889003,0.063282,0.064158,0.008122,0.109312,0.890688,0.063716,0.032920,0.499576,0.064080,-0.000852,962.6
164890007,0.109081,0.013205,0.008122,0.109312,0.890688,0.023548,0.011917,0.499952,0.110114,-0.000322,1653.8


,Precision,Recall,Accurracy,Hamming Loss,Hamming Acc,F1-Score,Jaccard (IoU),ROC AUC,Avg Precision AUC,MCC,Support
6374002,0.009945,0.084473,0.008122,0.109312,0.890688,0.017791,0.008977,0.512156,0.007397,0.008519,107.2
10370003,0.019537,0.110488,0.008122,0.109312,0.890688,0.033200,0.016884,0.504181,0.018284,0.003663,271.8
17338001,0.017085,0.004470,0.008122,0.109312,0.890688,0.007079,0.003554,0.499022,0.023919,-0.003674,359.6
39732003,0.074427,0.003622,0.008122,0.109312,0.890688,0.006905,0.003467,0.500029,0.077174,-0.000143,1158.6
47665007,0.021254,0.022175,0.008122,0.109312,0.890688,0.021685,0.010964,0.503019,0.015806,0.005851,235.4
59118001,0.067198,0.057674,0.008122,0.109312,0.890688,0.062055,0.032023,0.502855,0.061285,0.006151,915.0
59931005,0.052497,0.040541,0.008122,0.109312,0.890688,0.045738,0.023409,0.500131,0.052291,0.000257,785.0
111975006,0.020341,0.013048,0.008122,0.109312,0.890688,0.015878,0.008018,0.498645,0.024801,-0.003502,372.4
164889003,0.063282,0.064158,0.008122,0.109312,0.890688,0.063716,0.032920,0.499576,0.064080,-0.000852,962.6
164890007,0.109081,0.013205,0.008122,0.109312,0.890688,0.023548,0.011917,0.499952,0.110114,-0.000322,1653.8


,Precision,Recall,Accurracy,Hamming Loss,Hamming Acc,F1-Score,Jaccard (IoU),ROC AUC,Avg Precision AUC,MCC,Support
6374002,0.009945,0.084473,0.008122,0.109312,0.890688,0.017791,0.008977,0.512156,0.007397,0.008519,107.2
10370003,0.019537,0.110488,0.008122,0.109312,0.890688,0.033200,0.016884,0.504181,0.018284,0.003663,271.8
17338001,0.017085,0.004470,0.008122,0.109312,0.890688,0.007079,0.003554,0.499022,0.023919,-0.003674,359.6
39732003,0.074427,0.003622,0.008122,0.109312,0.890688,0.006905,0.003467,0.500029,0.077174,-0.000143,1158.6
47665007,0.021254,0.022175,0.008122,0.109312,0.890688,0.021685,0.010964,0.503019,0.015806,0.005851,235.4
59118001,0.067198,0.057674,0.008122,0.109312,0.890688,0.062055,0.032023,0.502855,0.061285,0.006151,915.0
59931005,0.052497,0.040541,0.008122,0.109312,0.890688,0.045738,0.023409,0.500131,0.052291,0.000257,785.0
111975006,0.020341,0.013048,0.008122,0.109312,0.890688,0.015878,0.008018,0.498645,0.024801,-0.003502,372.4
164889003,0.063282,0.064158,0.008122,0.109312,0.890688,0.063716,0.032920,0.499576,0.064080,-0.000852,962.6
164890007,0.109081,0.013205,0.008122,0.109312,0.890688,0.023548,0.011917,0.499952,0.110114,-0.000322,1653.8


,Precision,Recall,Accurracy,Hamming Loss,Hamming Acc,F1-Score,Jaccard (IoU),ROC AUC,Avg Precision AUC,MCC,Support
6374002,0.009945,0.084473,0.008122,0.109312,0.890688,0.017791,0.008977,0.512156,0.007397,0.008519,107.2
10370003,0.019537,0.110488,0.008122,0.109312,0.890688,0.033200,0.016884,0.504181,0.018284,0.003663,271.8
17338001,0.017085,0.004470,0.008122,0.109312,0.890688,0.007079,0.003554,0.499022,0.023919,-0.003674,359.6
39732003,0.074427,0.003622,0.008122,0.109312,0.890688,0.006905,0.003467,0.500029,0.077174,-0.000143,1158.6
47665007,0.021254,0.022175,0.008122,0.109312,0.890688,0.021685,0.010964,0.503019,0.015806,0.005851,235.4
59118001,0.067198,0.057674,0.008122,0.109312,0.890688,0.062055,0.032023,0.502855,0.061285,0.006151,915.0
59931005,0.052497,0.040541,0.008122,0.109312,0.890688,0.045738,0.023409,0.500131,0.052291,0.000257,785.0
111975006,0.020341,0.013048,0.008122,0.109312,0.890688,0.015878,0.008018,0.498645,0.024801,-0.003502,372.4
164889003,0.063282,0.064158,0.008122,0.109312,0.890688,0.063716,0.032920,0.499576,0.064080,-0.000852,962.6
164890007,0.109081,0.013205,0.008122,0.109312,0.890688,0.023548,0.011917,0.499952,0.110114,-0.000322,1653.8
